In [56]:
import numpy as np
import json
import os
import pandas as pd
from plotly import graph_objects as go
import matplotlib.pyplot as plt

In [18]:
#Toma los archivos .json del directorio y los une a una base de datos usando pandas.

data_list = []
base_dir = 'tech---ops/'

for file in os.listdir(base_dir):
    if 'json' in file:
        json_path = os.path.join(base_dir, file)
        json_data = pd.read_json(json_path, dtype=False)
        data_list.append(json_data)

df = pd.concat(data_list)


In [19]:
# Se agregaron las columnas nuevas segun las reacciones y ademas se ordenaron por el index.

nuevas_columnas_reacciones = ['t_finalizado', 't_revisado', 't_resuelto']

df_columnas_reaccion = df.reindex(columns=df.columns.tolist() + nuevas_columnas_reacciones)
df_columnas_reaccion['reactions'].fillna('', inplace=True)
df_columnas_reaccion = df_columnas_reaccion.reset_index(drop=True)



In [20]:
def merge_dict(lista):
    """Funci'on que une diccionarios en uno solo a travez de listas

    Args:
        lista (list): toma como argumdento una lista no vacia!!! que contenga varios diccionarios
    """
    new_dict = {'name':[], 'users':[]}

    for elemento in lista:
        new_dict['name'].append(elemento['name'])
        new_dict['users'].append(elemento['users'])

    return new_dict

In [21]:
# Se creo una lista vacia para almacenar todos los diccionarios de la columna reactions

lista_dict = []

# Se itero la columna reactions para almacenar los respectivos diccionarios usando la funcion anteriormente creada en lista_dict para despues generar un DataFrame con los datos

for i in df_columnas_reaccion.index:
    if df_columnas_reaccion['reactions'][i] != '':
        lista_dict.append(merge_dict(df_columnas_reaccion['reactions'][i]))
    else:
        lista_dict.append({'name': [], 'users': []})

df_reaction_provisoria = pd.DataFrame(lista_dict)

In [22]:
# Se creo una lista con los simbolos de reaccion

simbolos_reaccion = ['white_check_mark', 'eyes', 'raised_hands']

# Se itero la base de datos con el fin de almacenar True's & False's en las respectivas columnas de su reaccion

for i in df_reaction_provisoria.index:
    if simbolos_reaccion[0] in df_reaction_provisoria['name'][i]:
        df_columnas_reaccion['t_finalizado'][i] = True

for i in df_reaction_provisoria.index:
    if simbolos_reaccion[1] in df_reaction_provisoria['name'][i]:
        df_columnas_reaccion['t_revisado'][i] = True

for i in df_reaction_provisoria.index:
    if any(simbolos_reaccion[2] in s for s in df_reaction_provisoria['name'][i]):
        df_columnas_reaccion['t_resuelto'][i] = True

df_columnas_reaccion[['t_finalizado', 't_revisado', 't_resuelto']] = df_columnas_reaccion[['t_finalizado', 't_revisado', 't_resuelto']].fillna(value=False)

/tmp/ipykernel_243/2172078230.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_columnas_reaccion['t_finalizado'][i] = True
/tmp/ipykernel_243/2172078230.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_columnas_reaccion['t_revisado'][i] = True
/tmp/ipykernel_243/2172078230.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_columnas_reaccion['t_resuelto'][i] = True


In [23]:
# Seleccionamos las columnas utiles del df y ademas se elimnaron las filas duplicadas y se ordenaron segun la fecha y hora de la creacion de primer mensaje. 

df_clean = df_columnas_reaccion[['client_msg_id', 'text', 'user', 'ts', 'thread_ts', 'latest_reply', 't_finalizado', 't_revisado', 't_resuelto']].drop_duplicates().sort_values(['ts'])

df_clean = df_clean[df_clean['ts'] >= '1642171431.027300'].reset_index(drop=True)

In [24]:
# Columnas nuevas a agregar

nuevas_columnas = ['nuevo_req', 'ticket_ops', 'ticket_tech', 'urgente']

df_clean_new_columns = df_clean.reindex(columns=df_clean.columns.tolist() + nuevas_columnas)
df_clean_new_columns['latest_reply'].fillna('', inplace=True)

In [25]:
# Se creo una lista con los simbolos asociado a las nuevas columnas.

simbolos = [':writing_hand:', ':dart:', ':robot_face:',':rotating_light:']

# Se itero el df para asociarle un valor de True or False a las respectivas columnas segun la simbologia del mensaje principal del thread.

for i in df_clean_new_columns.index:
    if df_clean_new_columns['latest_reply'][i] != '':
            for j in range(len(simbolos)):
                if simbolos[j] in df_clean_new_columns['text'][i]:
                    df_clean_new_columns[df_clean_new_columns.columns[9 + j]][i] = True
                else:
                    df_clean_new_columns[df_clean_new_columns.columns[9 + j]][i] = False 

/tmp/ipykernel_243/3757963238.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_new_columns[df_clean_new_columns.columns[9 + j]][i] = False
/tmp/ipykernel_243/3757963238.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_new_columns[df_clean_new_columns.columns[9 + j]][i] = True


In [26]:
# Mostrar las primeras 10 filas del DataFrame

df_clean_new_columns.head(10)

,client_msg_id,text,user,ts,thread_ts,latest_reply,t_finalizado,t_revisado,t_resuelto,nuevo_req,ticket_ops,ticket_tech,urgente
0,85bc5361-aa7a-45c7-931c-b185d28298e2,*Resumen Simbología:*\n\n_*UPDATE:*_ \n _DEFIN...,U022N5F6ZGC,1642171431.027300,NaN,,False,False,False,NaN,NaN,NaN,NaN
1,89523989-32a4-4e4c-83ce-fed7c7c5f673,<@U02TWSY0V3M> tengo una duda con el mensaje d...,U02AJP0M7L1,1642173967.028200,1642089212.013900,,False,False,False,NaN,NaN,NaN,NaN
2,f97780ee-fcc0-48e7-9648-ec84f6d86413,"Si, dejamos este espacio para que allí como in...",U02TWSY0V3M,1642174239.028400,1642089212.013900,,False,False,False,NaN,NaN,NaN,NaN
3,3a1562c2-9dab-4cbd-b76b-a8b731c70c70,<@U02TWSY0V3M> ticket resuelto,U02AJP0M7L1,1642189849.029300,1642089212.013900,,True,False,True,NaN,NaN,NaN,NaN
4,a35833cc-50d2-4c28-ab9f-b5daad99a7c2,Gracias!,U02TWSY0V3M,1642192564.029700,1642089212.013900,,False,False,False,NaN,NaN,NaN,NaN
5,af871dcb-14ae-407f-aa91-f9712bcd7267,:rotating_light: :robot_face: *WOM-GEO*: BOT n...,U022N5F6ZGC,1642426940.032900,1642426940.032900,1642434618.042400,True,True,True,False,False,True,True
6,3c26f5b3-8526-4eae-b671-7e986fafe170,Notas: Agrego imágenes del JSON al cliente y d...,U022N5F6ZGC,1642426948.033000,1642426940.032900,,False,False,False,NaN,NaN,NaN,NaN
7,9336b3ea-2f96-4e24-b0f9-a8d89e84a79c,BBDD,U022N5F6ZGC,1642426968.033200,1642426940.032900,,False,False,False,NaN,NaN,NaN,NaN
8,acb98f9b-296b-47f3-bee5-023a33129192,Payload:,U022N5F6ZGC,1642426981.033600,1642426940.032900,,False,False,False,NaN,NaN,NaN,NaN
9,62047fe9-14d9-40a8-a327-080859145a0b,"{'status': 200, 'textResponse': '{""msj"":""OK"",""...",U022N5F6ZGC,1642428412.034100,1642426940.032900,,False,False,False,NaN,NaN,NaN,NaN


In [27]:
# Cambiar el nombre de la columna user a Owner

df_clean_new_columns = df_clean_new_columns.rename(columns = {'user': 'Owner'}, inplace=False)

In [28]:
df_clean_new_columns.head(10)

,client_msg_id,text,Owner,ts,thread_ts,latest_reply,t_finalizado,t_revisado,t_resuelto,nuevo_req,ticket_ops,ticket_tech,urgente
0,85bc5361-aa7a-45c7-931c-b185d28298e2,*Resumen Simbología:*\n\n_*UPDATE:*_ \n _DEFIN...,U022N5F6ZGC,1642171431.027300,NaN,,False,False,False,NaN,NaN,NaN,NaN
1,89523989-32a4-4e4c-83ce-fed7c7c5f673,<@U02TWSY0V3M> tengo una duda con el mensaje d...,U02AJP0M7L1,1642173967.028200,1642089212.013900,,False,False,False,NaN,NaN,NaN,NaN
2,f97780ee-fcc0-48e7-9648-ec84f6d86413,"Si, dejamos este espacio para que allí como in...",U02TWSY0V3M,1642174239.028400,1642089212.013900,,False,False,False,NaN,NaN,NaN,NaN
3,3a1562c2-9dab-4cbd-b76b-a8b731c70c70,<@U02TWSY0V3M> ticket resuelto,U02AJP0M7L1,1642189849.029300,1642089212.013900,,True,False,True,NaN,NaN,NaN,NaN
4,a35833cc-50d2-4c28-ab9f-b5daad99a7c2,Gracias!,U02TWSY0V3M,1642192564.029700,1642089212.013900,,False,False,False,NaN,NaN,NaN,NaN
5,af871dcb-14ae-407f-aa91-f9712bcd7267,:rotating_light: :robot_face: *WOM-GEO*: BOT n...,U022N5F6ZGC,1642426940.032900,1642426940.032900,1642434618.042400,True,True,True,False,False,True,True
6,3c26f5b3-8526-4eae-b671-7e986fafe170,Notas: Agrego imágenes del JSON al cliente y d...,U022N5F6ZGC,1642426948.033000,1642426940.032900,,False,False,False,NaN,NaN,NaN,NaN
7,9336b3ea-2f96-4e24-b0f9-a8d89e84a79c,BBDD,U022N5F6ZGC,1642426968.033200,1642426940.032900,,False,False,False,NaN,NaN,NaN,NaN
8,acb98f9b-296b-47f3-bee5-023a33129192,Payload:,U022N5F6ZGC,1642426981.033600,1642426940.032900,,False,False,False,NaN,NaN,NaN,NaN
9,62047fe9-14d9-40a8-a327-080859145a0b,"{'status': 200, 'textResponse': '{""msj"":""OK"",""...",U022N5F6ZGC,1642428412.034100,1642426940.032900,,False,False,False,NaN,NaN,NaN,NaN


In [29]:
# Se creo el DataFrame para los users

data_list = []
base_dir = 'user-data/'

for file in os.listdir(base_dir):
    if 'json' in file:
        json_path = os.path.join(base_dir, file)
        json_data = pd.read_json(json_path, dtype=False)
        data_list.append(json_data)

df_users = pd.concat(data_list)
df_users = df_users[['id', 'name']]

In [30]:
# Cambiar el nombre de la columna id a Owner

df_users = df_users.rename(columns = {'id': 'Owner'}, inplace=False)

In [31]:
# Combinar el df de user con el df original para obtener los nombres de cada mensaje

data_clean1 = pd.merge(df_users, df_clean_new_columns, on='Owner', how='outer')

data_clean1 = data_clean1[data_clean1['name'].notna()]
data_clean1 = data_clean1[data_clean1['latest_reply'] != ''].sort_values(['ts']).reset_index(drop=True)

In [32]:
data_clean1.head()

,Owner,name,client_msg_id,text,ts,thread_ts,latest_reply,t_finalizado,t_revisado,t_resuelto,nuevo_req,ticket_ops,ticket_tech,urgente
0,U022N5F6ZGC,istuardo,af871dcb-14ae-407f-aa91-f9712bcd7267,:rotating_light: :robot_face: *WOM-GEO*: BOT n...,1642426940.032900,1642426940.032900,1642434618.042400,True,True,True,False,False,True,True
1,U022N5F6ZGC,istuardo,66eee977-e3b1-47cf-84a1-e633c6202c7f,:rotating_light: :dart: *PY:* Proyectos con mú...,1642428966.036600,1642428966.036600,1642687425.075900,True,True,True,False,True,False,True
2,U022N5F6ZGC,istuardo,563c6f74-fd33-4a66-a584-48eecf1a9354,:rotating_light: :dart: *PY:* En algunos casos...,1642429006.037300,1642429006.037300,1642442773.044300,True,True,False,False,True,False,True
3,U02TWSY0V3M,paula,df6ea799-28c3-4f00-98dd-18a7820d30de,:rotating_light: :dart:Ajustar base de datos d...,1642431077.040300,1642431077.040300,1643118481.004400,True,True,False,False,True,False,True
4,U02TWSY0V3M,paula,70b063ce-6705-437e-ba1b-c060bf969175,:rotating_light: :robot_face:Enviar hoy a las ...,1642431296.041200,1642431296.041200,1642446093.045100,True,True,True,False,False,True,True


In [33]:
# Se tranformo de Unix TimeStamp a Pandas Datetime las columnas ts, thread_ts y latest_reply

data_clean1[['ts', 'thread_ts', 'latest_reply']] = data_clean1[['ts', 'thread_ts', 'latest_reply']].apply(pd.to_datetime, unit='s')
data_clean1.head()

,Owner,name,client_msg_id,text,ts,thread_ts,latest_reply,t_finalizado,t_revisado,t_resuelto,nuevo_req,ticket_ops,ticket_tech,urgente
0,U022N5F6ZGC,istuardo,af871dcb-14ae-407f-aa91-f9712bcd7267,:rotating_light: :robot_face: *WOM-GEO*: BOT n...,2022-01-17 13:42:20.032900094,2022-01-17 13:42:20.032900094,2022-01-17 15:50:18.042399883,True,True,True,False,False,True,True
1,U022N5F6ZGC,istuardo,66eee977-e3b1-47cf-84a1-e633c6202c7f,:rotating_light: :dart: *PY:* Proyectos con mú...,2022-01-17 14:16:06.036600113,2022-01-17 14:16:06.036600113,2022-01-20 14:03:45.075900078,True,True,True,False,True,False,True
2,U022N5F6ZGC,istuardo,563c6f74-fd33-4a66-a584-48eecf1a9354,:rotating_light: :dart: *PY:* En algunos casos...,2022-01-17 14:16:46.037300110,2022-01-17 14:16:46.037300110,2022-01-17 18:06:13.044300079,True,True,False,False,True,False,True
3,U02TWSY0V3M,paula,df6ea799-28c3-4f00-98dd-18a7820d30de,:rotating_light: :dart:Ajustar base de datos d...,2022-01-17 14:51:17.040299892,2022-01-17 14:51:17.040299892,2022-01-25 13:48:01.004400015,True,True,False,False,True,False,True
4,U02TWSY0V3M,paula,70b063ce-6705-437e-ba1b-c060bf969175,:rotating_light: :robot_face:Enviar hoy a las ...,2022-01-17 14:54:56.041199923,2022-01-17 14:54:56.041199923,2022-01-17 19:01:33.045099974,True,True,True,False,False,True,True


In [40]:
# Agregar nueva columna para encontrar el tiempo que estuvo abierto cada ticket.

data_clean1['Duracion_Ticket_horas'] = (data_clean1.latest_reply - data_clean1.ts) / pd.Timedelta(hours=1)

data_clean1 = data_clean1[['Owner','name','client_msg_id','text','ts','thread_ts','latest_reply','t_revisado','t_resuelto','t_finalizado','nuevo_req','ticket_ops','ticket_tech','urgente','Duracion_Ticket_horas']]

In [41]:
# Funnel entre tickets revisados, resueltos y finalizados

fig = go.Figure(go.Funnel(
    y = data_clean1.columns[7:10] ,
    x = [data_clean1.t_revisado.value_counts().loc[True], data_clean1.t_resuelto.value_counts().loc[True], data_clean1.t_finalizado.value_counts().loc[True]]))

fig.show()

In [91]:
# Tarjeta que muestra la duracion promedio de los tickets recibidos en Slack

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = data_clean1.describe()['Duracion_Ticket_horas']['mean'],
    number = {'suffix' : ' Horas'},
    title = {"text": "<span style='font-size:1.5em'>Duracion Promedio Tickets</span><br><span style='font-size:1.5em'></span><br>"},
    domain = {'x': [1, 1], 'y': [1, 1]}))

fig.show()

In [90]:
# Muestra la cantidad Total de Tickets Recibidos

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = data_clean1.describe()['Duracion_Ticket_horas']['count'],
    title = {"text": "<span style='font-size:1.5em'>Cantidad de Tickets</span><br><span style='font-size:1.5em'></span><br>"},
    domain = {'x': [1, 1], 'y': [1, 1]}))

fig.show()

In [37]:
# Exportando la data a excel en formato csv, ya que, en formato xlsx no permite exportarlo por el formato de las fechas

data_clean1.to_csv('data_slack_clean.csv')

In [87]:
# Descripcion analitica de la columna Duracion_tickets_horas

data_clean1.describe()

,Duracion_Ticket_horas
count,65.000000
mean,60.613052
std,67.194520
min,0.001944
25%,3.072501
50%,23.995059
75%,96.011594
max,313.569539
